In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import datetime
import os
import re
import docx

from langchain.document_loaders import UnstructuredWordDocumentLoader
from langchain.schema.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language
import urllib.request

from models.load_utils import save_docs_to_jsonl

In [ ]:

path_dir  = '../data/raw/tnt/'
output_dir = '../data/split/tnt/'
docxs = []

chunk_size = 2000
chunk_overlap = 200
length_function = len
today = datetime.today().strftime('%Y-%m-%d')

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if not os.path.exists(path_dir):
    os.makedirs(path_dir)

In [ ]:
file_data = []
file_names = os.listdir(path_dir)
for file in file_names:
    docxs.append(path_dir + file)
for doc in docxs:
    loader = UnstructuredWordDocumentLoader(doc, mode="elements")
    data = []
    data = loader.load()
    file_data.extend(data)
file_data






In [ ]:
def get_path(dir, title):
    # path_components = urlparse(url).path.split('/')
    return os.path.join(dir, re.sub('[^a-zA-Z0-9]', '-', title).lower()+'.pdf')

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=length_function,
)

In [ ]:
docs = []
for url, title, first_page in pdfs:
    path = get_path(path_dir, title)
    print(f"Processing {url} {path}")
    # download file if it doesn't exist
    if not os.path.exists(path):
        response = urllib.request.urlretrieve(url, path)
        print(response)
        print(f"Downloaded {url}")
    # load file
    loader = (path)
    pages = []
    for page in loader.load_and_split():
        page_number = page.metadata['page']+1
        if page_number < first_page:
            continue
        page.metadata["url"] = url
        page.metadata["title"] = f"{title} - {page_number}"
        pages.append(page)
    print(f"Processed {len(pages)} pages")
    splits = splits = text_splitter.split_documents(pages)
    print(f"Created {len(splits)} splits")
    for ix, split in enumerate(splits[:10]):
        print(ix, split.metadata["url"], split.metadata["title"])
        print(split.page_content)
        print("\n!!! SPLIT !!!\n")
    docs.extend(splits)

In [ ]:
len(docs)

In [ ]:
filename = os.path.join(output_dir, f"{today}.jsonl")
save_docs_to_jsonl(docs, filename)